# 神经网络

在pytorch中，可以使用`torch.nn`包来构建神经网络。

现在我们已经了解什么是`autograd`，`nn`包依赖于`autograd`去完成建模以及求微分。一个`nn.Module`包含了神经网络中的层，它还有一个方法`forware(input)`用于获得模型的输出。

下面是一个用于分类手写数字的网络：
![mnist](./images/003/mnist.png)
<center>图1.1 LeNet</center>

上面是一个很简单的前馈网络。它获得输入，然后沿着网络层往前传递，最后得到输出。

训练一个神经网络的主要步骤如下：

- 定义一个具有可学习参数(或称为权重)的神经网络
- 开始在数据集上迭代
- 将数据输入网络
- 计算loss
- 将loss反向传播
- 更新可学习的网络参数，通常使用算法：`weight = weight - learning_rate * gradient`

# 定义神经网络

下面就来定义一个神经网络：

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


在定义网络时，我们只需定义前向传播的过程，例如上面的`forward`函数，反向传播会由`autograd`包自动计算。在`forward`函数中你可以对`Tensor`进行任何操作。

可使用`net.parameters()`获得可学习的网络参数：

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


来试一下 $32 \times 32$ 的随机输入。**注意：此网络(LeNet)的输入大小为 $32 \times 32$，如果要使用MNIST数据集，需要将图片大小转换为 $32 \times 32$。**

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0675,  0.1205,  0.0065,  0.0294,  0.0679, -0.0201, -0.0432,  0.0713,
          0.0204, -0.0401]], grad_fn=<AddmmBackward>)


清除网络中的梯度缓存，然后使用一组随机值进行反向传播：

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div style="background-color:#e6e0f5;padding:10px;border-radius:5px;">
<span style="font-size:14px;color:white;background-color:#0ccbea;padding:5px;display:block;">NOTE</span>

`torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.

If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.
</div>

**回顾：**

- `torch.Tensor` - 一个多维数组，支持自动微分，并且在自动微分过程中保存关于张量的梯度。
- `nn.Module` - 神经网络模型，便于封装模型，而且有预定义方法支持移动到GPU计算，导出导入模型等。
- `nn.Parameter` - 一个`Tensor`，用于定义模型的参数(例如`weight`或`bias`)，定义式会自动加入到`Module`中。
- `autograd.Function` - 实现了一些预定义函数(例如`relu`)的前向和反向传播过程。

任何一个张量运算都会创建一个`Function`节点，并且这个节点会创建一个输出张量，张量运算也会被记录到模型的运算历史中。

**目前已学习的内容**：

- 定义网络
- 输入并反向传播

**剩余内容**：

- 计算loss
- 迭代更新网络的权重参数

# Loss 函数

Loss函数接收`(output,target)`输入对，然后计算出模型输出和目标值的偏离程度。

`nn`包中有几种不同的[loss函数](https://pytorch.org/docs/stable/nn.html)。例如最简单的`nn.MSELoss`计算`output`和`target`之间的平方误差。

例如：

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7537, grad_fn=<MseLossBackward>)


现在，如果跟踪loss的反向传播过程，通过它的`.grad_fn`属性，可以看到类似如下的计算图：
```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

当我们调用`loss.backward()`的时候，整个计算图的每个节点都会计算关于`loss`的微分。计算图中所有属性`requires_grad=True`的`Tensor`，它们的`.grad`属性中都会累积关于`loss`的梯度。

下面是回溯了几步的结果：

In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# 反向传播

进行反向传播很简单，只需调用`loss.backward()`。在调用之前需要清除已经累积的梯度，否则新计算的梯度会累加到已经存在的梯度当中。

现在调用`loss.backward()`，然后观察`conv1`的`bias`的梯度有什么变化：

In [10]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0023,  0.0104, -0.0071,  0.0076, -0.0022,  0.0197])


The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is [here](https://pytorch.org/docs/stable/nn.html).

# 更新网络参数

最简单的方法是随机梯度下降(Stochastic Gradient Descent, SGD)：

$$
\text{weight} = \text{weight} - \text{learning_rate} * \text{gradient}
$$

简单实现：
```python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```

但是框架有很多内置的优化方法，例如SGD, Nesterov-SGD, Adam, RMSProp等等。我们不需要自己写，`torch.optim`中包含了所有的优化方法，使用很简单：

In [11]:
import torch.optim as optim

In [13]:
# 创建优化器
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 一次迭代
optimizer.zero_grad()               # 清除梯度缓存
output = net(input)
loss = criterion(output, target)    # 计算loss
loss.backward()                     # 反向传播
optimizer.step()                    # 更新权重

<div style="background-color:#e6e0f5;padding:10px;border-radius:5px;">
<span style="font-size:14px;color:white;background-color:#0ccbea;padding:5px;display:block;">NOTE</span>
Observe how gradient buffers had to be manually set to zero using `optimizer.zero_grad()`. This is because gradients are accumulated as explained in <a href="https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#backprop">Backprop</a> section.
</div>